# Numerical integration

Numerical integration is a strategy to calculate the area under a curve numerically, without the need for the analytical solution of an integral. The can be very useful for complex and multidimensional problems, for example the area under a 3D LC-HRMS feature. In this case an analytical solution, implies that we first need to fully model the signal, then calculate the integral of the signal based on the model. Finally, we can use that integral to calculate the area under the peak. As you can imagine, this extremely difficult to achieve. Let's go through this together.

## E1: 

Let's generate a mass domain peak with area (A) of 100, m/z value (*``\mu``*) of 150.05, and a mass precision (*``\sigma``*) of 0.02. 

$$ f(x,A,\mu,\sigma) =  \frac{A}{\sigma \sqrt{2 \pi}}\exp{\frac{-(x - \mu)^2}{2\sigma^{2}}} $$

In [1]:
using DataSci4Chem

f(x) = (A/(s * sqrt(2 * pi))) * exp.(-(x .- m).^2 ./ (2*s^2))

x = collect(149:0.002:151)
A = 100
m = 150.05
s = 0.02

y = f(x)

plot(x,y,label = "simulated signal",xlabel = "m/z values", ylabel = "intensity")

## E2: 

Let's define the window where we want to integrate this peak. How can we do that?

> **Answer to E2:** 
>
> Here given that we are working with very clean simulated signal, we can choose values above a certain threshold for example 1.

In [ ]:

y2intg = y[y .>1] # 
x2intg = x[y .>1]

plot(x,y,label = "simulated signal",xlabel = "m/z values", ylabel = "intensity")
plot!(x2intg,y2intg,label = "selected signal")

## E3: 

What is the easiest way to estimate the area under this peak? 

> **Answer to E3:** 
>
> We can use a rectangle around this peak to perform the integration. 

In [ ]:
plot(x2intg,y2intg,label = "selected signal")
plot!([x2intg[1],x2intg[1]],[0,maximum(y2intg)],lc = :black, lw = 3, label =false)
plot!([x2intg[end],x2intg[end]],[0,maximum(y2intg)],lc = :black, lw = 3, label =false)
plot!([x2intg[1],x2intg[end]],[maximum(y2intg),maximum(y2intg)],lc = :black, lw = 3, label ="inetgration area")

## E4:

We have the rectangle. Now how do we do the integration? 

> **Answer to E4:**
>
> The area of a rectangle is calculated by multiplying the length (*l*) by the width (*w*). 
> $$ area_{rec} = l \times w = \frac{A}{\sigma \sqrt{2 \pi}} \times \Delta m/z $$

In [ ]:
d_mz = x2intg[end] - x2intg[1]
inten = maximum(y2intg)

area_rec(l,w) = l .* w

area_rec(d_mz,inten)

## E5:

What if we only know the edges of the integration zone? Is there a way to find the *l* parameter automatically? 

> **Answer to E5:**
>
> Assuming symmetry we can find the *l* as the *f(x)*, where the *x* is the midpoint of the integration window. 

## E6: 

How do you find the midpoint? 

> **Answer to E6:**
>
> The midpoint can be calculated by averaging the edges of the integration area. 

In [ ]:
mdp = mean([x2intg[end],x2intg[1]])
y_mdp = f(mdp)
scatter!([mdp],[y_mdp],label = "midpoint")

## E7: 

Why cannot we use the maximum function for this?

> **Answer to E7:**
>
> For more complex systems the max may not work as it may be at the edge of the integration window. 

## E8: 

What is wrong with this estimation of the area?

> **Answer to E8:**
>
> As it can be seen, our method greatly overestimates the area under the peak. 

## E9:

How can we solve this problem?

> **Answer to E9:**
>
> We can increase the number of the rectangles used for the integration, for example two. 

In [ ]:
plot(x2intg,y2intg,label = "selected signal")
plot!([x2intg[1],x2intg[1]],[0,maximum(y2intg)],lc = :black, lw = 3, label =false)
plot!([x2intg[end],x2intg[end]],[0,maximum(y2intg)],lc = :black, lw = 3, label =false)
plot!([mdp,mdp],[0,maximum(y2intg)],lc = :black, lw = 3, label =false)
plot!([x2intg[1],x2intg[end]],[maximum(y2intg),maximum(y2intg)],lc = :black, lw = 3, label ="inetgration area")

In [ ]:
mdp2 = mean([x2intg[1],mdp])
mdp3 = mean([x2intg[end],mdp])

y2 = f(mdp2)
y3 = f(mdp3)

plot(x2intg,y2intg,label = "selected signal")
plot!([x2intg[1],x2intg[1]],[0,y2],lc = :black, lw = 3, label =false)
plot!([x2intg[end],x2intg[end]],[0,y2],lc = :black, lw = 3, label =false)
plot!([mdp,mdp],[0,y2],lc = :black, lw = 3, label =false)
plot!([x2intg[1],x2intg[end]],[y2,y3],lc = :black, lw = 3, label = "integration area")

scatter!([mdp2,mdp3],[y2,y3],label = "midpoints with two boxes")

## E10:

How can we automatically calculate the midpoints, for any number of boxes?

In [ ]:
n_w = 3

w_t = x2intg[end] - x2intg[1]

w_i = w_t / n_w

edges = collect(x2intg[1]:(w_i):x2intg[end])

mdps = edges[1:end-1] .+ w_i/2
 


In [ ]:
plot(x2intg,y2intg,label = "selected signal")
plot!([edges[1],edges[2]],[f(mdps[1]),f(mdps[1])],lc = :black, lw = 3, label = "box 1")
plot!([edges[2],edges[3]],[f(mdps[2]),f(mdps[2])],lc = :black, lw = 3, label = "box 2")
plot!([edges[3],edges[4]],[f(mdps[3]),f(mdps[3])],lc = :black, lw = 3, label = "box 3")
scatter!(mdps,f(mdps),label = "midpoints")

## E11: 

Can we write a function to calculate the midpoints based on the window and the number of boxes?

In [ ]:
function midpoint_calc(a,b,n_w)
    w_t = b - a

    w_i = w_t / n_w

    edges = collect(a:(w_i):b)

    mdps = edges[1:end-1] .+ w_i/2

    return(edges,mdps)

end 

In [ ]:
n_w = 10 
edges,mdps = midpoint_calc(x2intg[1],x2intg[end],n_w)

p = plot(x2intg,y2intg,label = "selected signal")
scatter!(p,mdps,f(mdps,A,m,s),label = "midpoints")

for i = 1:length(mdps)
    plot!(p,[edges[i],edges[i+1]],[f(mdps[i]),f(mdps[i])],lc = :black, lw = 3, label = "box $i")
end 

p

## E12: 

How can we calculate the area under the curve?

> **Answer to E12:** 
>
> We need to sum up the areas of all the boxes to get an estimation of the area under the curve.
> $$ \int_{a}^{b} f(x)dx \approx \sum_{i=1}^{n} f(\frac{x_i + x_{i+1}}{2})\Delta x $$

## E13: 

Calculate the area under the peak for number of windows varying from two to 50.

In [ ]:
est_int = zeros(49)

for i =1:length(est_int)
    edges,mdps = midpoint_calc(x2intg[1],x2intg[end],2 + (i-1))
    dx = edges[2] - edges[1]
    l = f(mdps,A,m,s)
    est_int[i] = sum(l .* dx)
end 

p1 = scatter(2:50,est_int,label="midpoint method",xlabel="# windows",ylabel="estimated area")
p2 = scatter(2:50,est_int,label=false,xlabel="# windows",ylabel="estimated area")
xlims!(2.5,10.5)

plot(p1,p2)

## E14: 

As you can see, for number of windows larger than 6 we do not see a significant change in the estimated area. Can we use this for deciding how many windows we should use?

> **Answer to E14:**
>
> Theoretically, you can estimate the area for several window numbers and generate a plot to what you see above. However, for more complex systems, that may be very expensive, computationally. Therefore, usually a large number of windows (e.g. 100) is set as default value.

## E15: 

Can we write a function to perform midpoint integration? 

> **Answer to E15:**
>
> There is a function called "*midpoint_integration(-)*" implemented within this package that does this type of analysis for you.

## E16:

Is there another way to perform numerical integration?

> **Answer to E16:**
>
> We can use a trapezoid rather than a rectangle to solve this problem.

## E17: 

How do you calculate the area of a trapezoid?

> **Answer to E17:**
>
> $$ area_{trap} = \frac{l + w}{2} \times h $$

## E18:

What does this remind you of? 

> **Answer to E18:**
>
> This is the midpoint solution to the area of a rectangle.

## E19: 

How would this look for two trapezoids?

> **Answer to E19:**
>
> $$ area_{total} = \frac{f(x_0)+f(x_{1})}{2} \times \Delta x + \frac{f(x_1)+f(x_{2})}{2} \times \Delta x $$ 
>
> $$ area_{total} = \frac{\Delta x}{2} (f(x_0) + 2f(x_1) + f(x_2)) $$
>
> In other words the points at the edge of the window is multiplied by one while all the other points are multiplied by two.

## E20: 

Let's visually compare the two approaches to each other for two windows.

In [ ]:
n_w = 2
edges,mdps = midpoint_calc(x2intg[1],x2intg[end],n_w)

p = plot(x2intg,y2intg,label = "selected signal")
scatter!(p,mdps,f(mdps,A,m,s),label = "midpoints")

for i = 1:length(mdps)
    plot!(p,[edges[i],edges[i+1]],[f(mdps[i]),f(mdps[i])],lc = :black, lw = 3, label = "box $i")
    plot!(p,[edges[i],edges[i+1]],[f(edges[i]),f(edges[i+1])],lc = :red, lw = 3, label = " trapezoid box $i")
end 
plot!(p,[edges[2],edges[2]],[0,f(edges[2])],label=false,lc=:blue)

p

In [ ]:
a = x2intg[1]
b = x2intg[end]

trap_int = DataSci4Chem.trapezoid_rule(f, a, b; n = 2)
error_trap = A - trap_int

In [ ]:
midp_int = DataSci4Chem.midpoint_integration(f, a, b; n_wins =2)
error_midp = A - midp_int

## E21: 

Write a function to perform numerical integration via trapezoid rule.

> **Answer to E21:**
>
> An implementation of trapezoid rule in the package is available. 